# Predicting outcome of chess games based on my performance this year

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

from keras.models import Sequential
from keras.layers import Dense

from time import time

from keras.wrappers.scikit_learn import KerasClassifier

from dummies_bins_test_train_cv import get_Xy_train_test

Using TensorFlow backend.


## If new data is added, uncomment and run the next line

In [2]:
# from clean_chess_game_log import main_cleanup
# _,_,_ = main_cleanup('../data/dest.pgn')

## Load the data to run predictions

In [2]:
df = pd.read_csv('../data/use_for_predictions.csv')

## Create Train and Test sets, along with clean the data
### Also, see the shape to know the data is in right shape

In [15]:
X_train, X_test, y_train, y_test, X, y, df_clean = get_Xy_train_test(
    df, .98, .99)

y Shape: (2104,)
.
X Shape: (2104, 29)
X_train Shape: (2064, 29)
X_test Shape: (40, 29)
y_train Shape: (2064,)
y_test Shape: (40,)


## Making sure the data is in the correct format

In [16]:
X_train = X_train.astype('float64')
X_test = X_test.astype('float64')
y_train = y_train.astype('int64')
y_test = y_test.astype('int64')

## Scale the data for easier computations

In [17]:
std_sclr = StandardScaler()
X_train = std_sclr.fit_transform(X_train)
X_test = std_sclr.fit_transform(X_test)

### See the scale of each value if interested

In [18]:
std_sclr.scale_

array([65.38348415,  0.494343  ,  8.91175488,  1.        ,  1.        ,
        1.        ,  0.26339134,  0.21794495,  0.3799671 ,  1.        ,
        0.21794495,  0.26339134,  0.21794495,  0.15612495,  0.4       ,
        0.26339134,  0.3       ,  0.26339134,  1.        ,  0.15612495,
        0.15612495,  1.        ,  1.        ,  0.4       ,  0.33071891,
        0.33071891,  0.4       ,  0.3       ,  0.3       ])

## Creating the classifier:
__The _Fun_ Part!!__

### Ran the classifier with new data every time I played on Chess.com.

Which is how I ended up with the following steps:

- __Step 1__
    - Add the sequential layer
- __Step 2__
    - Add the __Dense__ layer with following parameters:
        - Units = 128
        - Initializer = Uniform
        - Activation = Softmax (Since the outcome is either 1 or 0, Softmax works wonders)
        - Input Dim = the second part of the X Shape (# of columns)
- __Step 3__
    - Add the second __Dense__ layer with following parameters:
        - Units = 256 (Double it up!!)
        - Initializer = Normal (Throw a curve ball!)
        - Activation = ReLu (Prevents Vanashing Gradiant problem)
- __Step 4__
    - Add the third __Dense__ layer with following parameters:
        - Units = 64 (Make it easier for the compiling)
        - Initializer = Uniform
        - Activation = Softmax
- __Step 5__
    - Add the final __Dense__ layer with following parameters:
        - Units = 1 (Just need 1 column for output)
        - Initializer = Normal (#MuscleConfusion. I kid!)
        - Activation = Sigmoid
- __Step 6__
    - Only think left is to __COMPILE__ it. From all the parameters I've tested. These work the best:
        - Optimizer = Nadam (Nesterov Adam optimizer - Nadam is Adam RMSprop with Nesterov momentum)
        - loss = Mean Absolute Error (Didn't get great results with binary_crossentropy, even though it was my first choice. Mean Squared Error, made error very small, since the loss is between 0 and 1.)
        - metrics = Binary Accuracy (Only have 0 or 1 to predict)

In [19]:
def _classifier():
    classifier = Sequential()

    classifier.add(Dense(units=128, kernel_initializer='uniform',
                         activation='softmax', input_dim=X.shape[1]))

    classifier.add(
        Dense(units=256, kernel_initializer='normal', activation='relu'))

    classifier.add(
        Dense(units=64, kernel_initializer='uniform', activation='softmax'))

    classifier.add(
        Dense(units=1, kernel_initializer='normal', activation='sigmoid'))

    classifier.compile(
        optimizer='nadam', loss='mae', metrics=['accuracy'])

    return classifier

## Fitting the training data

### I wanted to test different batch sizes to get the results on the real games. 

I started testing fitting classifier __For Loop__ with the following parameters:

1. Batch Size:
    1. 20
    2. 16
    3. 12
    4. 8
    5. 6
2. Epochs:
    1. 50
    2. 100
    3. 200

In [11]:
results = {}

In [12]:
for bs in [20, 16, 12, 8, 6]:
    for ep in [50, 100, 200]:
        classifier = _classifier()
        classifier.fit(X_train, y_train, batch_size=bs, epochs=ep,
                       class_weight='balanced', shuffle=False)
        y_pred = classifier.predict(X_test)
        y_pred = (y_pred > 0.5)
        cm = confusion_matrix(y_test, y_pred)
        results[(f'bs:{bs}, ep:{ep} (cm)')] = cm
        results[(f'bs:{bs}, ep:{ep} (accuracy)')
                ] = f'{((cm[0][0]+cm[1][1])/cm.sum()*100).round(1)}%'
        print(results)

Epoch 1/10
2074/2074 [==============================] - 1s 533us/step - loss: 0.5001 - acc: 0.4990
Epoch 2/10
2074/2074 [==============================] - 1s 243us/step - loss: 0.4994 - acc: 0.5058
Epoch 3/10
2074/2074 [==============================] - 1s 342us/step - loss: 0.4658 - acc: 0.6726
Epoch 4/10
2074/2074 [==============================] - 1s 249us/step - loss: 0.4232 - acc: 0.6967
Epoch 5/10
2074/2074 [==============================] - 1s 259us/step - loss: 0.3973 - acc: 0.7112
Epoch 6/10
2074/2074 [==============================] - 1s 242us/step - loss: 0.3813 - acc: 0.7088
Epoch 7/10
2074/2074 [==============================] - 1s 290us/step - loss: 0.3670 - acc: 0.7093
Epoch 8/10
2074/2074 [==============================] - 1s 261us/step - loss: 0.3547 - acc: 0.7189
Epoch 9/10
2074/2074 [==============================] - 1s 253us/step - loss: 0.3432 - acc: 0.7208
Epoch 10/10
2074/2074 [==============================] - 1s 280us/step - loss: 0.3340 - acc: 0.7242
{'bs:20, 

2074/2074 [==============================] - 1s 258us/step - loss: 0.2411 - acc: 0.7594
Epoch 74/100
2074/2074 [==============================] - 0s 230us/step - loss: 0.2388 - acc: 0.7633
Epoch 75/100
2074/2074 [==============================] - 0s 232us/step - loss: 0.2377 - acc: 0.7628
Epoch 76/100
2074/2074 [==============================] - 0s 219us/step - loss: 0.2392 - acc: 0.7618
Epoch 77/100
2074/2074 [==============================] - 0s 223us/step - loss: 0.2355 - acc: 0.7652
Epoch 78/100
2074/2074 [==============================] - 0s 241us/step - loss: 0.2391 - acc: 0.7623
Epoch 79/100
2074/2074 [==============================] - 0s 239us/step - loss: 0.2362 - acc: 0.7642
Epoch 80/100
2074/2074 [==============================] - 0s 227us/step - loss: 0.2364 - acc: 0.7647
Epoch 81/100
2074/2074 [==============================] - 0s 226us/step - loss: 0.2362 - acc: 0.7652
Epoch 82/100
2074/2074 [==============================] - 0s 217us/step - loss: 0.2373 - acc: 0.7642
Epo

Epoch 131/200
2074/2074 [==============================] - 0s 224us/step - loss: 0.2409 - acc: 0.7599
Epoch 132/200
2074/2074 [==============================] - 0s 215us/step - loss: 0.2446 - acc: 0.7560
Epoch 133/200
2074/2074 [==============================] - 0s 228us/step - loss: 0.2449 - acc: 0.7555
Epoch 134/200
2074/2074 [==============================] - 0s 217us/step - loss: 0.2382 - acc: 0.7623
Epoch 135/200
2074/2074 [==============================] - 0s 224us/step - loss: 0.2361 - acc: 0.7637
Epoch 136/200
2074/2074 [==============================] - 0s 230us/step - loss: 0.2318 - acc: 0.7686
Epoch 137/200
2074/2074 [==============================] - 0s 232us/step - loss: 0.2339 - acc: 0.7666
Epoch 138/200
2074/2074 [==============================] - 0s 237us/step - loss: 0.2343 - acc: 0.7657
Epoch 139/200
2074/2074 [==============================] - 0s 223us/step - loss: 0.2391 - acc: 0.7608
Epoch 140/200
2074/2074 [==============================] - 0s 225us/step - loss: 0

2074/2074 [==============================] - 1s 283us/step - loss: 0.2471 - acc: 0.7570
Epoch 42/50
2074/2074 [==============================] - 1s 279us/step - loss: 0.2474 - acc: 0.7560
Epoch 43/50
2074/2074 [==============================] - 1s 270us/step - loss: 0.2468 - acc: 0.7570
Epoch 44/50
2074/2074 [==============================] - 1s 289us/step - loss: 0.2458 - acc: 0.7580
Epoch 45/50
2074/2074 [==============================] - 1s 268us/step - loss: 0.2462 - acc: 0.7560
Epoch 46/50
2074/2074 [==============================] - 1s 274us/step - loss: 0.2446 - acc: 0.7584
Epoch 47/50
2074/2074 [==============================] - 1s 275us/step - loss: 0.2456 - acc: 0.7565
Epoch 48/50
2074/2074 [==============================] - 1s 294us/step - loss: 0.2439 - acc: 0.7589
Epoch 49/50
2074/2074 [==============================] - 1s 288us/step - loss: 0.2481 - acc: 0.7546
Epoch 50/50
2074/2074 [==============================] - 1s 279us/step - loss: 0.2481 - acc: 0.7541
{'bs:20, ep:

2074/2074 [==============================] - 1s 288us/step - loss: 0.2370 - acc: 0.7633
Epoch 66/100
2074/2074 [==============================] - 1s 364us/step - loss: 0.2335 - acc: 0.7671
Epoch 67/100
2074/2074 [==============================] - 1s 289us/step - loss: 0.2410 - acc: 0.7594
Epoch 68/100
2074/2074 [==============================] - 1s 279us/step - loss: 0.2472 - acc: 0.7541
Epoch 69/100
2074/2074 [==============================] - 1s 293us/step - loss: 0.2408 - acc: 0.7594
Epoch 70/100
2074/2074 [==============================] - 1s 286us/step - loss: 0.2428 - acc: 0.7584
Epoch 71/100
2074/2074 [==============================] - 1s 316us/step - loss: 0.2416 - acc: 0.7594
Epoch 72/100
2074/2074 [==============================] - 1s 286us/step - loss: 0.2371 - acc: 0.7642
Epoch 73/100
2074/2074 [==============================] - 1s 296us/step - loss: 0.2367 - acc: 0.7642
Epoch 74/100
2074/2074 [==============================] - 1s 306us/step - loss: 0.2341 - acc: 0.7657
Epo

...

2074/2074 [==============================] - 1s 444us/step - loss: 0.2473 - acc: 0.7527
Epoch 104/200
2074/2074 [==============================] - 1s 462us/step - loss: 0.2410 - acc: 0.7589
Epoch 105/200
2074/2074 [==============================] - 1s 448us/step - loss: 0.2432 - acc: 0.7565
Epoch 106/200
2074/2074 [==============================] - 1s 441us/step - loss: 0.2480 - acc: 0.7522
Epoch 107/200
2074/2074 [==============================] - 1s 437us/step - loss: 0.2551 - acc: 0.7449
Epoch 108/200
2074/2074 [==============================] - 1s 432us/step - loss: 0.2541 - acc: 0.7459
Epoch 109/200
2074/2074 [==============================] - 1s 461us/step - loss: 0.2487 - acc: 0.7512
Epoch 110/200
2074/2074 [==============================] - 1s 423us/step - loss: 0.2449 - acc: 0.7555
Epoch 111/200
2074/2074 [==============================] - 1s 456us/step - loss: 0.2510 - acc: 0.7488
Epoch 112/200
2074/2074 [==============================] - 1s 441us/step - loss: 0.2510 - acc: 0

2074/2074 [==============================] - 1s 694us/step - loss: 0.2814 - acc: 0.7237
Epoch 17/25
2074/2074 [==============================] - 1s 686us/step - loss: 0.2833 - acc: 0.7218
Epoch 18/25
2074/2074 [==============================] - 1s 657us/step - loss: 0.2783 - acc: 0.7261
Epoch 19/25
2074/2074 [==============================] - 1s 665us/step - loss: 0.2770 - acc: 0.7252
Epoch 20/25
2074/2074 [==============================] - 1s 716us/step - loss: 0.2776 - acc: 0.7252
Epoch 21/25
2074/2074 [==============================] - 1s 698us/step - loss: 0.2733 - acc: 0.7295
Epoch 22/25
2074/2074 [==============================] - 1s 678us/step - loss: 0.2760 - acc: 0.7257
Epoch 23/25
2074/2074 [==============================] - 1s 654us/step - loss: 0.2688 - acc: 0.7334
Epoch 24/25
2074/2074 [==============================] - 1s 652us/step - loss: 0.2746 - acc: 0.7261
Epoch 25/25
2074/2074 [==============================] - 1s 628us/step - loss: 0.2708 - acc: 0.7300
{'bs:20, ep:

Epoch 1/100
2074/2074 [==============================] - 3s 1ms/step - loss: 0.4946 - acc: 0.5376
Epoch 2/100
2074/2074 [==============================] - 1s 645us/step - loss: 0.4217 - acc: 0.7001
Epoch 3/100
2074/2074 [==============================] - 1s 642us/step - loss: 0.3808 - acc: 0.7001
Epoch 4/100
2074/2074 [==============================] - 2s 819us/step - loss: 0.3554 - acc: 0.7054
Epoch 5/100
2074/2074 [==============================] - 2s 852us/step - loss: 0.3425 - acc: 0.7030
Epoch 6/100
2074/2074 [==============================] - 2s 867us/step - loss: 0.3276 - acc: 0.7088
Epoch 7/100
2074/2074 [==============================] - 2s 941us/step - loss: 0.3172 - acc: 0.7126
Epoch 8/100
2074/2074 [==============================] - 1s 642us/step - loss: 0.3121 - acc: 0.7112
Epoch 9/100
2074/2074 [==============================] - 1s 690us/step - loss: 0.3115 - acc: 0.7064
Epoch 10/100
2074/2074 [==============================] - 1s 690us/step - loss: 0.3007 - acc: 0.7160
E

2074/2074 [==============================] - 1s 614us/step - loss: 0.2552 - acc: 0.7459
Epoch 42/200
2074/2074 [==============================] - 1s 620us/step - loss: 0.2599 - acc: 0.7411
Epoch 43/200
2074/2074 [==============================] - 1s 621us/step - loss: 0.2647 - acc: 0.7353
Epoch 44/200
2074/2074 [==============================] - 1s 602us/step - loss: 0.2722 - acc: 0.7295
Epoch 45/200
2074/2074 [==============================] - 1s 598us/step - loss: 0.2642 - acc: 0.7363
Epoch 46/200
2074/2074 [==============================] - 1s 609us/step - loss: 0.2595 - acc: 0.7411
Epoch 47/200
2074/2074 [==============================] - 1s 592us/step - loss: 0.2603 - acc: 0.7401
Epoch 48/200
2074/2074 [==============================] - 1s 609us/step - loss: 0.2708 - acc: 0.7285
Epoch 49/200
2074/2074 [==============================] - 1s 598us/step - loss: 0.2647 - acc: 0.7358
Epoch 50/200
2074/2074 [==============================] - 1s 601us/step - loss: 0.2634 - acc: 0.7372
Epo

2074/2074 [==============================] - 1s 612us/step - loss: 0.2352 - acc: 0.7652
Epoch 123/200
2074/2074 [==============================] - 1s 617us/step - loss: 0.2366 - acc: 0.7633
Epoch 124/200
2074/2074 [==============================] - 1s 597us/step - loss: 0.2308 - acc: 0.7695
Epoch 125/200
2074/2074 [==============================] - 1s 601us/step - loss: 0.2341 - acc: 0.7662
Epoch 126/200
2074/2074 [==============================] - 1s 595us/step - loss: 0.2391 - acc: 0.7599
Epoch 127/200
2074/2074 [==============================] - 1s 595us/step - loss: 0.2349 - acc: 0.7652
Epoch 128/200
2074/2074 [==============================] - 1s 592us/step - loss: 0.2367 - acc: 0.7633
Epoch 129/200
2074/2074 [==============================] - 1s 630us/step - loss: 0.2395 - acc: 0.7599
Epoch 130/200
2074/2074 [==============================] - 1s 596us/step - loss: 0.2395 - acc: 0.7604
Epoch 131/200
2074/2074 [==============================] - 1s 585us/step - loss: 0.2419 - acc: 0

### Here are the best & the worst combinations:

### The best:
1. batch Size = 12, Epochs = 50
    - CM = [__11__ 2][4 __13__]
    - Accuracy = 80%
2. batch Size = 12, Epochs = 100
    - CM = [__11__ 2][4 __13__]
    - Accuracy = 80%
3. batch Size = 20, Epochs = 100
    - CM = [__11__ 2][4 __13__]
    - Accuracy = 80%
4. batch Size = 8, Epochs = 50
    - CM = [10 3][4 __13__]
    - Accuracy = 76.67%

### The worst:
1. batch Size = 8, Epochs = 100
    - CM = [11 2] [__7__ 10]
    - Accuracy = 70%
2. batch Size = 16, Epochs = 50
    - CM = [10 3] [__6__ 11]
    - Accuracy = 70%

#### The rest were at 73.33%

NOTE:
- All batches of 6 were at 73%
- All Epochs of 200 were at 73%

*Need to run K-fold to verify results*

In [20]:
kfdict = {}

## K-Fold Cross Validation
Following parameters selected from the test above (batch_size: epochs):

__6: [50, 100]<br>
8 : [50, 100]<br>
12 : [50, 100]<br>
16 : [50, 100]<br>
20 : [50, 100]__

In [21]:
for bs in [20, 16, 12, 8, 6]:
    for ep in [50, 100]:
        st = time()
        classifier = KerasClassifier(_classifier, batch_size=bs, epochs=ep, verbose=4)
        kfmodel = cross_val_score(classifier, X_train, y_train,
                                  cv=7, n_jobs=-2, scoring='accuracy', verbose=0)
        kfdict[(f'bs:{bs}, ep:{ep} (mean & std)')] = [round(kfmodel.mean(), 2), round(kfmodel.std(), 3)]
        kfdict[(f'bs:{bs}, ep:{ep} (CV)')] = kfmodel.round(2)
        print(kfdict)
        print(time()-st)

Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50
Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/5

Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78/100
Epoch 79/100
Epoch 80/100
Epoch 81/100
Epoc

KeyboardInterrupt: 